# Telco Churn Analysis

Importing libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.ticker as mtick
import matplotlib.pyplot as plt
%matplotlib inline

**Dataset Info:**
Sample Data Set containing Telco customer data and showing customers left last month

In [ ]:
telco_base_data = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

Look at first 5 record of data

In [ ]:
telco_base_data.head()

checking the attributes of datatypes

In [ ]:
telco_base_data.shape

In [ ]:
telco_base_data.columns.values

In [ ]:
telco_base_data.dtypes

SeniorCitizen is actually a categorical hence the 25%-50%-75% distribution is not propoer

75% customers have tenure less than 55 months

Average Monthly charges are USD 64.76 whereas 25% customers pay more than USD 89.85 per month

In [ ]:
telco_base_data.describe()

In [ ]:
telco_base_data['Churn'].value_counts().plot(kind='barh', figsize=(8, 6))
plt.xlabel("count", labelpad=14)
plt.ylabel("Target variable", labelpad=14)
plt.title("count of Target varible per categgory", y=1.02);

In [ ]:
100*telco_base_data['Churn'].value_counts()/len(telco_base_data['Churn'])

In [ ]:
telco_base_data['Churn'].value_counts()

* Data is highly imbalanced, ratio = 73:27<br>
* So we analyse the data with other features while taking the target values separately to get some insights.

In [ ]:
telco_base_data.info(verbose = True)

In [ ]:
missing=pd.DataFrame((telco_base_data.isnull().sum())*100/telco_base_data.shape[0]).reset_index()
plt.figure(figsize=(16,5))
ax = sns.pointplot(x='index',y=0,data=missing)
plt.xticks(rotation = 90, fontsize=7)
plt.title("percentage of missing values")
plt.ylabel("percentage")
plt.show()

## Data Cleaning


**1.** Create a copy of base data for manupulation & processing

In [ ]:
telco_data = telco_base_data.copy()

**2.** Total Charges should be numeric amount. Let's convert it to numerical data type

In [ ]:
telco_data['TotalCharges'] = pd.to_numeric(telco_data['TotalCharges'], errors='coerce')
telco_data.isnull().sum()

**3.** As we can see there are 11 missing values in TotalCharges column. Let's check these records 

In [ ]:
telco_data.loc[telco_data ['TotalCharges'].isnull()==True]

**4. Missing Value Treatement**

Since the % of these records compared to total dataset is very low ie 0.15%, it is safe to ignore them from further processing.

In [ ]:
telco_data.dropna(how='any', inplace=True)

**5.** Divide customers into bins based on tenure e.g. for tenure < 12 months: assign a tenure group if 1-12, for tenure between 1 to 2 Yrs, tenure group of 13-24; so on...

In [ ]:
print(telco_data['tenure'].max())

In [ ]:
labels=["{0} - {1}".format(i, i+11) for i in range (1,72,12)]
telco_data['tenure_group']=pd.cut(telco_data.tenure, range(1,80,12), right=False, labels=labels) 

In [ ]:
telco_data['tenure_group'].value_counts()

**6.** Remove columns not required for processing

In [ ]:
telco_data.drop(columns=['customerID','tenure'], axis=1, inplace=True)
telco_data.head()

## Data Exploration
**1.** Plot distibution of individual predictors by churn

### Univariate Analysis

In [ ]:
for i,predictor in enumerate(telco_data.drop(columns=['Churn','TotalCharges','MonthlyCharges'])):
    plt.figure(i)
    sns.countplot(data=telco_data, x=predictor, hue='Churn')

**2.** Convert the target variable 'Churn'  in a binary numeric variable i.e. Yes=1 ; No = 0

In [ ]:
telco_data['Churn']=np.where(telco_data.Churn == 'Yes',1,0)

In [ ]:
telco_data.head()

**3.** Convert all the categorical variables into dummy variables

In [ ]:
telco_data_dumies= pd.get_dummies(telco_data)
telco_data_dumies.head()

Relationship between Monthly anc Total charges

In [ ]:
sns.lmplot(data=telco_data_dumies, x='MonthlyCharges', y='TotalCharges',fit_reg=False)

Total Charges increase as Monthly Charges increase - as expected.

Churn by monthly and total charges

In [ ]:
Mth=sns.kdeplot(telco_data_dumies.MonthlyCharges[(telco_data_dumies["Churn"] == 0) ], color="Red",shade=True)
Mth=sns.kdeplot(telco_data_dumies.MonthlyCharges[(telco_data_dumies["Churn"] == 1) ], ax=Mth, color="Blue",shade=True)
Mth.legend(["NO Churn","Churn"],loc='upper right')
Mth.set_ylabel('Density')
Mth.set_xlabel('Monthly Charges')
Mth.set_title('Monthly charges by churn')

**Insight:** Churn is high when Monthly Charges ar high

In [ ]:
Tot=sns.kdeplot(telco_data_dumies.TotalCharges[(telco_data_dumies["Churn"] == 0) ], color="Red",shade=True)
Tot=sns.kdeplot(telco_data_dumies.TotalCharges[(telco_data_dumies["Churn"] == 1) ], ax=Tot, color="Blue",shade=True)
Tot.legend(["NO Churn","Churn"],loc='upper right')
Tot.set_ylabel('Density')
Tot.set_xlabel('Total Charges')
Tot.set_title('Total charges by churn')

**Surprising insight ** as higher Churn at lower Total Charges

However if we combine the insights of 3 parameters i.e. Tenure, Monthly Charges & Total Charges then the picture is bit clear :- Higher Monthly Charge at lower tenure results into lower Total Charge. Hence, all these 3 factors viz **Higher Monthly Charge**,  **Lower tenure** and **Lower Total Charge** are linkd to **High Churn**.

 Build a corelation of all predictors with 'Churn'

In [ ]:
plt.figure(figsize=(20,8))
telco_data_dumies.corr()['Churn'].sort_values(ascending=False).plot(kind='barh')

**Derived Insight: **

**HIGH** Churn seen in case of  **Month to month contracts**, **No online security**, **No Tech support**, **First year of subscription** and **Fibre Optics Internet**

**LOW** Churn is seens in case of **Long term contracts**, **Subscriptions without internet service** and **The customers engaged for 5+ years**

Factors like **Gender**, **Availability of PhoneService** and **# of multiple lines** have alomost **NO** impact on Churn

This is also evident from the **Heatmap** below

In [ ]:
plt.figure(figsize=(12,12))
sns.heatmap(telco_data_dumies.corr(), cmap="Paired")

### Bivariate Analysis

In [ ]:
new_df1_target0=telco_data.loc[telco_data["Churn"]==0]
new_df1_target1=telco_data.loc[telco_data["Churn"]==1]

In [ ]:
def uniplot(df,col,title,hue =None):
    
    sns.set_style('whitegrid')
    sns.set_context('talk')
    plt.rcParams["axes.labelsize"] = 20
    plt.rcParams['axes.titlesize'] = 22
    plt.rcParams['axes.titlepad'] = 30
    
    
    temp = pd.Series(data = hue)
    fig, ax = plt.subplots()
    width = len(df[col].unique()) + 7 + 4*len(temp.unique())
    fig.set_size_inches(width , 8)
    plt.xticks(rotation=45)
    plt.yscale('log')
    plt.title(title)
    ax = sns.countplot(data = df, x= col, order=df[col].value_counts().index,hue = hue,palette='bright') 
        
    plt.show()

In [ ]:
uniplot(new_df1_target1,col='Partner',title='Distribution of Gender for Churned Customers',hue='gender')

In [ ]:
uniplot(new_df1_target0,col='Partner',title='Distribution of Gender for Non Churned Customers',hue='gender')

In [ ]:
uniplot(new_df1_target1,col='PaymentMethod',title='Distribution of PaymentMethod for Churned Customers',hue='gender')

In [ ]:
uniplot(new_df1_target1,col='Contract',title='Distribution of Contract for Churned Customers',hue='gender')

In [ ]:
uniplot(new_df1_target1,col='TechSupport',title='Distribution of TechSupport for Churned Customers',hue='gender')

In [ ]:
uniplot(new_df1_target1,col='SeniorCitizen',title='Distribution of SeniorCitizen for Churned Customers',hue='gender')

# CONCLUSION

These are some of the quick insights from this exercise:

1. Electronic check medium are the highest churners
2. Contract Type - Monthly customers are more likely to churn because of no contract terms, as they are free to go customers.
3. No Online security, No Tech Support category are high churners
4. Non senior Citizens are high churners


In [ ]:
telco_data_dumies.to_csv('tel_churn.csv',index=False)